In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model

# Define paths to your train and validation folders
train_data_dir = '/Users/farhanisrak/cvpr/farhan/train'
validation_data_dir = '/Users/farhanisrak/cvpr/farhan/validation'

# Define image size and other parameters
img_width, img_height = 224, 224
batch_size = 32

# Preprocess and augment data using ImageDataGenerator
train_datagen = ImageDataGenerator(
    rescale=1.0/255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True
)

validation_datagen = ImageDataGenerator(rescale=1.0/255)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical'
)

validation_generator = validation_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical'
)

# Create the ResNet-50 base model
base_model = ResNet50(weights='imagenet', include_top=False)

# Add custom layers on top of the base model
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(2, activation='softmax')(x)  # Two classes: '1' and '2'

model = Model(inputs=base_model.input, outputs=predictions)

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
num_epochs = 2
steps_per_epoch = train_generator.n // train_generator.batch_size
validation_steps = validation_generator.n // validation_generator.batch_size

model.fit(
    train_generator,
    steps_per_epoch=steps_per_epoch,
    epochs=num_epochs,
    validation_data=validation_generator,
    validation_steps=validation_steps
)

# Save the trained model
model.save('your_face_recognition_model.keras')


Found 811 images belonging to 2 classes.
Found 811 images belonging to 2 classes.
Found 143 images belonging to 2 classes.
Found 143 images belonging to 2 classes.
Epoch 1/2
Epoch 1/2
25/25 [==============================] - 129s 5s/step - loss: 0.2846 - accuracy: 0.9487 - val_loss: 76.5035 - val_accuracy: 0.7188
Epoch 2/2
Epoch 2/2
25/25 [==============================] - 121s 5s/step - loss: 1.5791e-04 - accuracy: 1.0000 - val_loss: 32.2886 - val_accuracy: 0.7109


In [2]:
# Compile the model
model.compile(optimizer='sgd', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
import cv2
import numpy as np
from keras.models import load_model
from keras.preprocessing.image import img_to_array
from tensorflow.keras.applications.vgg16 import preprocess_input

# Load the trained model
model.compile(optimizer='sgd', loss='categorical_crossentropy', metrics=['accuracy'])


# Define dimensions for resizing and preprocessing input images
input_shape = (224, 224)  # Adjust according to your model's input size

# Load the Haarcascades for face detection
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# Open the webcam
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    
    # Convert the frame to grayscale for face detection
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    # Detect faces in the frame
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))
    
    for (x, y, w, h) in faces:
        # Extract the face ROI
        face_roi = frame[y:y+h, x:x+w]
        
        # Preprocess the face image
        face_roi = cv2.resize(face_roi, input_shape)
        face_roi = img_to_array(face_roi)
        face_roi = preprocess_input(face_roi)
        face_roi = np.expand_dims(face_roi, axis=0)
        
        # Perform face recognition using the model
        predictions = model.predict(face_roi)
        
        # Assuming 'predictions' is an array of probabilities for different classes,
        # you can customize this part based on how your model outputs results.
        # For instance, if predictions[0][0] > threshold, then it's a match.
        # Otherwise, it's not a match.
        threshold = 0.5  # Adjust as needed
        
        if predictions[0][0] > threshold:
            cv2.putText(frame, "Entry Granted", (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)
        else:
            cv2.putText(frame, "No Entry", (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 0, 255), 2)
        
        # Draw a rectangle around the detected face
        cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 0, 0), 2)
    
    cv2.imshow("Face Recognition", frame)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


1/1 [==============================] - 0s 52ms/step


In [ ]:
import cv2
import numpy as np
from keras.models import load_model
from keras.optimizers import Adam  # Import the optimizer
from keras.preprocessing.image import img_to_array
from tensorflow.keras.applications.vgg16 import preprocess_input
import matplotlib.pyplot as plt

# Load the trained model architecture
model = load_model('your_face_recognition_model.keras')

# Recreate the optimizer
optimizer = Adam()  # You can adjust the optimizer's parameters if needed

# Compile the model with the recreated optimizer
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

# Define dimensions for resizing and preprocessing input images
input_shape = (224, 224)  # Adjust according to your model's input size

# Rest of the code remains the same...

# Plot the accuracy graph
plt.plot(accuracies)
plt.xlabel('Frame')
plt.ylabel('Accuracy (1=Granted, 0=Not Granted)')
plt.title('Face Recognition Accuracy')
plt.show()
